# Classification using pretrained, well-known models
This notebook aims to create a set of benchmarks for the project, using well-known, thoroughly studied models.
Either with pretrained weights, or training with new data.

In [6]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import master_data_functions.functions as dfunc
from keras.utils import to_categorical

# silence deprecation warnings from tensorflow
import warnings
warnings.simplefilter('ignore', FutureWarning)

# File import
# Sample filenames are:
# CeBr10kSingle_1.txt -> single events, 
# CeBr10kSingle_2.txt -> single events
# CeBr10k_1.txt -> mixed single and double events 
# CeBr10.txt -> small file of 10 samples

keywords = {
    "single_1": "CeBr10kSingle_1",
    "single_2": "CeBr10kSingle_2",
    "mix": "CeBr10k_1",
    "small": "CeBr10"}
    
dataset = "mix"
data = dfunc.import_data("sample")
images = data[keywords[dataset]]["images"]
energies = data[keywords[dataset]]["energies"]
positions = data[keywords[dataset]]["positions"]
labels = to_categorical(data[keywords[dataset]]["labels"])
n_classes = labels.shape[1]


print("Number of classes: {}".format(n_classes))
print("Energies shape: {}".format(energies.shape))
print("Positions shape: {}".format(positions.shape))
print("Labels shape: {}".format(labels.shape))
      

Number of classes: 2
Energies shape: (10000, 2)
Positions shape: (10000, 4)
Labels shape: (10000, 2)


In [2]:
# VGG16 expects 3 channels. Solving this by concatenating the image data 
# to itself, to form three identical channels

images = np.concatenate((images, images, images), axis=3)

print("Image data shape: {}".format(images.shape))

Image data shape: (10000, 16, 16, 3)



## Set up training and test data

In [3]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2)
print("Training and test data shapes:")
print("x_train: {}".format(x_train.shape))
print("x_test: {}".format(x_test.shape))
print("y_train: {}".format(y_train.shape))
print("y_test: {}".format(y_test.shape))




Training and test data shapes:
x_train: (8000, 16, 16, 3)
x_test: (2000, 16, 16, 3)
y_train: (8000, 2)
y_test: (2000, 2)


## VGG-16
Using Keras as our framework with Tensorflow backend

In [4]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

model = VGG16(weights='imagenet', include_top=False)


W0810 17:36:59.730076 140090784458368 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0810 17:36:59.761569 140090784458368 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0810 17:36:59.768360 140090784458368 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/notebooks/pretrained/pretrained_venv/lib64/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0810 17:36:59.853263 140090784458368 deprecation_wrapper.py:119] From /home/ulvik/git/master_analysis/n

In [5]:
# Parameters for the model
batch_size = 32
epochs = 10

model.evaluate(x_test, y_test, batch_size=batch_size)

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

## Predict on test-set and plot some examples
To compare predicted positions with actual positions. 

In [ ]:
predicted_pos = model.predict(x_test)

In [ ]:


# Plot some images, with electron origin positions
%matplotlib inline

index = 100
fig, ax = plt.subplots(3, 3, sharex='col', sharey='row', figsize=(12,12))

# Reshape test-data for plotting
x_plot = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2])
for i in range(3):
    for j in range(3):
        # plot image
        ax[i, j].imshow(x_plot[index + i*3 + j])
        
        # plot true origin of event
        x = y_test[index + i*3 + j][0]
        y = y_test[index + i*3 + j][1]
        ax[i, j].plot(x, y, 'rx')
        
        # plot predicted origin of event
        x_pred = predicted_pos[index + i*3 + j][0]
        y_pred = predicted_pos[index + i*3 + j][1]
        ax[i, j].plot(x_pred, y_pred, 'wx')
plt.show()